In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.read_csv('diabetes_cleaned_12-15-20.csv')

In [7]:
df.shape

(69970, 44)

In [75]:
df.columns

Index(['41_in_diag', '250_in_diag', '272_in_diag', '278_in_diag',
       '401_in_diag', '403_in_diag', '428_in_diag', '434_in_diag',
       '440_in_diag', '486_in_diag', '574_in_diag', '577_in_diag',
       '682_in_diag', '707_in_diag', '715_in_diag', '722_in_diag',
       '780_in_diag', '786_in_diag', 'A1Cresult_None', 'admission_source_id_5',
       'admission_source_id_7', 'admission_source_id_17',
       'admission_source_id_4', 'admission_type_id_3', 'admission_type_id_6',
       'age_num', 'diabetesMed_Yes', 'discharge_disposition_id_2',
       'discharge_disposition_id_3', 'discharge_disposition_id_4',
       'discharge_disposition_id_5', 'discharge_disposition_id_6',
       'discharge_disposition_id_18', 'discharge_disposition_id_22',
       'discharge_disposition_id_25', 'max_glu_serum_None', 'metformin_used',
       'number_diagnoses', 'number_emergency', 'number_inpatient',
       'num_lab_procedures', 'pioglitazone_used', 'time_in_hospital',
       'readmit_30d'],
      dty

In [3]:
from sklearn.model_selection import train_test_split

X = df.drop(columns='readmit_30d')
y = df.readmit_30d
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [41]:
#Lasso grid search
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

roc_scorer = make_scorer(roc_auc_score)
kf = KFold(n_splits=3,shuffle=True,random_state=5)
estimator = LogisticRegression(penalty='l1',solver='liblinear',random_state=5)

major_indices = y_train[y_train==0].index
minor_indices = y_train[y_train==1].index
sampled_indices = []
for i in range(42,92):
    sampled_indices.append(resample(y_train[major_indices],replace=False,n_samples=5022,random_state=i).index)
    
scaler_X = StandardScaler().fit(X_train)
#scaler_y = StandardScaler().fit(y_train.to_numpy().reshape(-1,1))
scaled_X = scaler_X.transform(X_train)
#scaled_y = scaler_y.transform(y_train.to_numpy().reshape(-1,1))
scaled_X = pd.DataFrame(scaled_X,index=X_train.index)
#scaled_y = pd.Series(scaled_y.flatten(),index=y_train.index)

vals = np.concatenate([np.linspace(100,10,10)[:-1],\
                       np.linspace(10,1,10)[:-1],\
                       np.linspace(1,.1,10)[:-1],\
                       np.linspace(.1,.01,10)[:-1],\
                       np.linspace(.01,.001,10)])
                       
params = {'C':vals}

results = []
for i in sampled_indices:
    X_balanced = pd.concat([scaled_X.loc[i],scaled_X.loc[minor_indices]])
    y_balanced = pd.concat([y_train[i],y_train[minor_indices]])
    selector = GridSearchCV(estimator=estimator, param_grid=params,cv=kf, scoring=roc_scorer, verbose=1,n_jobs=3)
    selector = selector.fit(X_balanced, y_balanced)
    results.append(selector.cv_results_['mean_test_score'])

Fitting 3 folds for each of 46 candidates, totalling 138 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  82 tasks      | elapsed:    4.3s
[Parallel(n_jobs=3)]: Done 138 out of 138 | elapsed:    6.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 46 candidates, totalling 138 fits


[Parallel(n_jobs=3)]: Done  82 tasks      | elapsed:    3.4s
[Parallel(n_jobs=3)]: Done 138 out of 138 | elapsed:    5.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 46 candidates, totalling 138 fits


...

[Parallel(n_jobs=3)]: Done  82 tasks      | elapsed:    5.2s
[Parallel(n_jobs=3)]: Done 138 out of 138 | elapsed:    7.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 46 candidates, totalling 138 fits


[Parallel(n_jobs=3)]: Done  82 tasks      | elapsed:    5.1s
[Parallel(n_jobs=3)]: Done 138 out of 138 | elapsed:    6.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 46 candidates, totalling 138 fits


[Parallel(n_jobs=3)]: Done  82 tasks      | elapsed:    3.7s
[Parallel(n_jobs=3)]: Done 138 out of 138 | elapsed:    5.2s finished


In [46]:
features = pd.DataFrame(np.concatenate([*results],axis=0).reshape((50,46)))
features.columns = vals
pd.concat([features.apply(lambda x: x.mean(),axis=0),features.apply(lambda x: x.std(),axis=0)],axis=1)

,0,1
100.000,0.604426,0.004018
90.000,0.604428,0.004019
80.000,0.604430,0.004021
70.000,0.604428,0.004022
60.000,0.604428,0.004024
50.000,0.604432,0.004025
40.000,0.604432,0.004020
30.000,0.604432,0.004019
20.000,0.604428,0.004022
10.000,0.604422,0.004027


In [38]:
#Ridge regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

roc_scorer = make_scorer(roc_auc_score)
kf = KFold(n_splits=3,shuffle=True,random_state=5)
estimator = LogisticRegression(penalty='l2',solver='liblinear',random_state=5)

major_indices = y_train[y_train==0].index
minor_indices = y_train[y_train==1].index
sampled_indices = []
for i in range(42,92):
    sampled_indices.append(resample(y_train[major_indices],replace=False,n_samples=5022,random_state=i).index)
    
scaler_X = StandardScaler().fit(X_train)
#scaler_y = StandardScaler().fit(y_train.to_numpy().reshape(-1,1))
scaled_X = scaler_X.transform(X_train)
#scaled_y = scaler_y.transform(y_train.to_numpy().reshape(-1,1))
scaled_X = pd.DataFrame(scaled_X,index=X_train.index)
#scaled_y = pd.Series(scaled_y.flatten(),index=y_train.index)

vals = np.concatenate([np.linspace(100,10,10)[:-1],\
                       np.linspace(10,1,10)[:-1],\
                       np.linspace(1,.1,10)[:-1],\
                       np.linspace(.1,.01,10)[:-1],\
                       np.linspace(.01,.001,10)[:-1],\
                       np.linspace(.001,.0001,10)[:-1],\
                       np.linspace(.0001,.00001,10)])
params = {'C':vals}

results = []
for i in sampled_indices:
    X_balanced = pd.concat([scaled_X.loc[i],scaled_X.loc[minor_indices]])
    y_balanced = pd.concat([y_train[i],y_train[minor_indices]])
    selector = GridSearchCV(estimator=estimator, param_grid=params,cv=kf, scoring=roc_scorer, verbose=1,n_jobs=3)
    selector = selector.fit(X_balanced, y_balanced)
    results.append(selector.cv_results_['mean_test_score'])

Fitting 3 folds for each of 64 candidates, totalling 192 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    9.9s
[Parallel(n_jobs=3)]: Done 187 out of 192 | elapsed:   15.2s remaining:    0.4s
[Parallel(n_jobs=3)]: Done 192 out of 192 | elapsed:   15.3s finished


Fitting 3 folds for each of 64 candidates, totalling 192 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  82 tasks      | elapsed:    3.4s
[Parallel(n_jobs=3)]: Done 192 out of 192 | elapsed:    6.7s finished


Fitting 3 folds for each of 64 candidates, totalling 192 fits


...

[Parallel(n_jobs=3)]: Done  82 tasks      | elapsed:    4.6s
[Parallel(n_jobs=3)]: Done 192 out of 192 | elapsed:    9.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 64 candidates, totalling 192 fits


[Parallel(n_jobs=3)]: Done  82 tasks      | elapsed:    4.7s
[Parallel(n_jobs=3)]: Done 192 out of 192 | elapsed:    8.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 64 candidates, totalling 192 fits


[Parallel(n_jobs=3)]: Done  82 tasks      | elapsed:    3.2s
[Parallel(n_jobs=3)]: Done 192 out of 192 | elapsed:    6.3s finished


In [40]:
features = pd.DataFrame(np.concatenate([*results],axis=0).reshape((50,64)))
features.columns = vals
features.apply(lambda x: x.mean(),axis=0)

100.00000    0.604420
90.00000     0.604420
80.00000     0.604420
70.00000     0.604420
60.00000     0.604422
50.00000     0.604422
40.00000     0.604420
30.00000     0.604420
20.00000     0.604420
10.00000     0.604418
9.00000      0.604420
8.00000      0.604422
7.00000      0.604422
6.00000      0.604424
5.00000      0.604424
4.00000      0.604422
3.00000      0.604418
2.00000      0.604428
1.00000      0.604428
0.90000      0.604432
0.80000      0.604430
0.70000      0.604434
0.60000      0.604434
0.50000      0.604436
0.40000      0.604442
0.30000      0.604436
0.20000      0.604438
0.10000      0.604445
0.09000      0.604463
0.08000      0.604459
0.07000      0.604493
0.06000      0.604515
0.05000      0.604519
0.04000      0.604563
0.03000      0.604637
0.02000      0.604702
0.01000      0.604924
0.00900      0.604943
0.00800      0.604989
0.00700      0.604955
0.00600      0.605045
0.00500      0.605177
0.00400      0.605186
0.00300      0.605396
0.00200      0.605456
0.00100   

In [4]:
#Lasso grid search coefs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

roc_scorer = make_scorer(roc_auc_score)
kf = KFold(n_splits=3,shuffle=True,random_state=5)
estimator = LogisticRegression(penalty='l1',solver='liblinear',random_state=5)

major_indices = y_train[y_train==0].index
minor_indices = y_train[y_train==1].index
sampled_indices = []
for i in range(42,142):
    sampled_indices.append(resample(y_train[major_indices],replace=False,n_samples=5022,random_state=i).index)
    
scaler_X = StandardScaler().fit(X_train)
#scaler_y = StandardScaler().fit(y_train.to_numpy().reshape(-1,1))
scaled_X = scaler_X.transform(X_train)
#scaled_y = scaler_y.transform(y_train.to_numpy().reshape(-1,1))
scaled_X = pd.DataFrame(scaled_X,index=X_train.index)
#scaled_y = pd.Series(scaled_y.flatten(),index=y_train.index)
                       
params = {'C':[0.7]}

results = []
for i in sampled_indices:
    X_balanced = pd.concat([scaled_X.loc[i],scaled_X.loc[minor_indices]])
    y_balanced = pd.concat([y_train[i],y_train[minor_indices]])
    selector = GridSearchCV(estimator=estimator, param_grid=params,cv=kf, scoring=roc_scorer, verbose=1,n_jobs=3)
    selector = selector.fit(X_balanced, y_balanced)
    results.append(selector.best_estimator_.coef_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    5.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished


In [60]:
features = pd.DataFrame(np.concatenate([*results],axis=0).reshape((50,43)))
features.columns = X_train.columns
pd.concat([features.apply(lambda x: x.mean(),axis=0),features.apply(lambda x: x.std(),axis=0)],axis=1).sort_values(0)

,0,1
786_in_diag,-0.086723,0.012941
admission_source_id_17,-0.065586,0.018182
722_in_diag,-0.058891,0.015822
admission_source_id_4,-0.052455,0.016202
max_glu_serum_None,-0.048490,0.015983
admission_source_id_7,-0.045600,0.020108
715_in_diag,-0.039993,0.017511
486_in_diag,-0.039868,0.014068
272_in_diag,-0.039662,0.015614
401_in_diag,-0.036112,0.014276


In [5]:
features = pd.DataFrame(np.concatenate([*results],axis=0).reshape((100,43)))
features.columns = X_train.columns
pd.concat([features.apply(lambda x: x.mean(),axis=0),features.apply(lambda x: x.std(),axis=0)],axis=1).sort_values(0)

,0,1
786_in_diag,-0.086773,0.012978
admission_source_id_17,-0.066007,0.019569
722_in_diag,-0.058318,0.014872
admission_source_id_4,-0.051611,0.015257
max_glu_serum_None,-0.047096,0.018114
admission_source_id_7,-0.042346,0.018965
715_in_diag,-0.038967,0.016089
272_in_diag,-0.038859,0.015195
486_in_diag,-0.037981,0.014232
401_in_diag,-0.037293,0.014026


In [6]:
df.readmit_30d.mean()

0.08970987566099757

In [7]:
df.groupby('number_inpatient').readmit_30d.mean()

number_inpatient
0     0.081176
1     0.128926
2     0.185210
3     0.237581
4     0.236842
5     0.284314
6     0.400000
7     0.473684
8     0.230769
9     0.428571
10    0.800000
11    0.500000
12    1.000000
Name: readmit_30d, dtype: float64

In [8]:
df.groupby('discharge_disposition_id_22').readmit_30d.mean()

discharge_disposition_id_22
0    0.086142
1    0.263307
Name: readmit_30d, dtype: float64

In [9]:
df.groupby('discharge_disposition_id_3').readmit_30d.mean()

discharge_disposition_id_3
0    0.083369
1    0.133880
Name: readmit_30d, dtype: float64

In [10]:
df.groupby('discharge_disposition_id_5').readmit_30d.mean()

discharge_disposition_id_5
0    0.088174
1    0.205915
Name: readmit_30d, dtype: float64

In [11]:
df.groupby('discharge_disposition_id_2').readmit_30d.mean()

discharge_disposition_id_2
0    0.088629
1    0.137752
Name: readmit_30d, dtype: float64

In [74]:
print(df.discharge_disposition_id_22.sum())
print(df.discharge_disposition_id_3.sum())
print(df.discharge_disposition_id_5.sum())
print(df.discharge_disposition_id_2.sum())


1409
8784
913
1539


In [78]:
dir(selector.best_estimator_)

['C',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_estimator_type',
 '_get_param_names',
 '_get_tags',
 '_predict_proba_lr',
 'class_weight',
 'classes_',
 'coef_',
 'decision_function',
 'densify',
 'dual',
 'fit',
 'fit_intercept',
 'get_params',
 'intercept_',
 'intercept_scaling',
 'l1_ratio',
 'max_iter',
 'multi_class',
 'n_iter_',
 'n_jobs',
 'penalty',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'random_state',
 'score',
 'set_params',
 'solver',
 'sparsify',
 'tol',
 'verbose',
 'warm_start']

In [82]:
pd.Series(selector.best_estimator_.decision_function(X_test)).sort_values()

3385      0.669233
765       1.241408
9951      1.317269
4827      1.393469
2946      1.435972
12933     1.441366
5443      1.441378
11278     1.453972
7424      1.540812
7897      1.839249
7262      1.844374
8713      1.888256
892       1.902634
847       1.928817
12938     1.929539
6578      1.946068
5881      1.979496
2510      1.991752
8364      2.030644
12263     2.049619
4955      2.072529
2402      2.106844
12380     2.116034
5340      2.135548
12802     2.186189
11389     2.214336
1600      2.287808
6005      2.288748
13589     2.333393
9084      2.337598
5800      2.341898
1815      2.363798
5533      2.364918
6155      2.379282
6611      2.399164
3571      2.410811
10772     2.411497
5270      2.413599
8835      2.432710
3711      2.432805
6895      2.453026
11316     2.487379
10644     2.521553
8590      2.523748
3939      2.527040
5043      2.530515
2083      2.531415
9720      2.536820
12214     2.548459
12926     2.552687
10498     2.555244
12672     2.561818
2672      2.

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

roc_scorer = make_scorer(roc_auc_score)
kf = KFold(n_splits=3,shuffle=True,random_state=5)
estimator = LogisticRegression(penalty='l1',solver='liblinear',random_state=5)

scaler_X = StandardScaler().fit(X_train)
#scaler_y = StandardScaler().fit(y_train.to_numpy().reshape(-1,1))
scaled_X = scaler_X.transform(X_train)
#scaled_y = scaler_y.transform(y_train.to_numpy().reshape(-1,1))
scaled_X = pd.DataFrame(scaled_X,index=X_train.index)
#scaled_y = pd.Series(scaled_y.flatten(),index=y_train.index)
estimator = LogisticRegression(C=0.7,penalty='l1',solver='liblinear',random_state=5)                 
params = {'C':[0.7]}
estimator.fit(X_train,y_train)
train_pred= estimator.predict(X_train)
test_pred = estimator.predict(X_test)
print(roc_auc_score(y_train,train_pred))
print(roc_auc_score(y_test,test_pred))

0.5016463573152486
0.5018350336779892


In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

roc_scorer = make_scorer(roc_auc_score)
kf = KFold(n_splits=3,shuffle=True,random_state=5)
estimator = LogisticRegression(penalty='l1',solver='liblinear',random_state=5)

major_indices = y_train[y_train==0].index
minor_indices = y_train[y_train==1].index
sampled_indices = []
for i in range(42,92):
    sampled_indices.append(resample(y_train[major_indices],replace=False,n_samples=5022,random_state=i).index)
    
scaler_X = StandardScaler().fit(X_train)
#scaler_y = StandardScaler().fit(y_train.to_numpy().reshape(-1,1))
scaled_X = scaler_X.transform(X_train)
#scaled_y = scaler_y.transform(y_train.to_numpy().reshape(-1,1))
scaled_X = pd.DataFrame(scaled_X,index=X_train.index)
#scaled_y = pd.Series(scaled_y.flatten(),index=y_train.index)
                       
params = {'C':[0.7]}

results = []
for i in sampled_indices:
    X_balanced = pd.concat([scaled_X.loc[i],scaled_X.loc[minor_indices]])
    y_balanced = pd.concat([y_train[i],y_train[minor_indices]])
    selector = GridSearchCV(estimator=estimator, param_grid=params,cv=kf, scoring=roc_scorer, verbose=1,n_jobs=3)
    selector = selector.fit(X_balanced, y_balanced)
    results.append(selector.predict_proba(X_test)[:,1])

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    7.7s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.5s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.2s finished


In [35]:
features = pd.DataFrame(np.concatenate([*results],axis=0).reshape((50,13994)))
tmp = pd.concat([features.apply(lambda x: x.mean(),axis=0),features.apply(lambda x: x.std(),axis=0)],axis=1)

In [30]:
tmp

,0,1
0,0.907387,0.080550
1,0.996479,0.004537
2,0.977603,0.017806
3,0.997501,0.003002
4,0.998682,0.002053
5,0.999034,0.001800
6,0.999352,0.001362
7,0.996291,0.004891
8,0.997186,0.003969
9,0.985669,0.012214


In [36]:
roc_auc_score(y_test,tmp[0])

0.5845339264654233

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

roc_scorer = make_scorer(roc_auc_score)
kf = KFold(n_splits=3,shuffle=True,random_state=5)
estimator = LogisticRegression(penalty='l2',solver='liblinear',random_state=5)

major_indices = y_train[y_train==0].index
minor_indices = y_train[y_train==1].index
sampled_indices = []
for i in range(42,92):
    sampled_indices.append(resample(y_train[major_indices],replace=False,n_samples=5022,random_state=i).index)
    
scaler_X = StandardScaler().fit(X_train)
#scaler_y = StandardScaler().fit(y_train.to_numpy().reshape(-1,1))
scaled_X = scaler_X.transform(X_train)
#scaled_y = scaler_y.transform(y_train.to_numpy().reshape(-1,1))
scaled_X = pd.DataFrame(scaled_X,index=X_train.index)
#scaled_y = pd.Series(scaled_y.flatten(),index=y_train.index)
                       
params = {'C':[0.002]}

results = []
for i in sampled_indices:
    X_balanced = pd.concat([scaled_X.loc[i],scaled_X.loc[minor_indices]])
    y_balanced = pd.concat([y_train[i],y_train[minor_indices]])
    selector = GridSearchCV(estimator=estimator, param_grid=params,cv=kf, scoring=roc_scorer, verbose=1,n_jobs=3)
    selector = selector.fit(X_balanced, y_balanced)
    results.append(selector.predict_proba(X_test)[:,1])

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.2s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.2s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.2s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.2s finished
